In [1]:
# pydantic data ko validate karnay k liye istimaal hota hai...

from datetime import datetime
from typing import Tuple

from pydantic import BaseModel


class Delivery(BaseModel):
    timestamp: datetime
    dimensions: Tuple[int, int]


m = Delivery(timestamp='2020-01-02T03:04:05Z', dimensions=['10', '20'])
print(repr(m.timestamp))
#> datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
print(m.dimensions)
#> (10, 20)


datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
(10, 20)


In [3]:
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int  
    name: str = 'John Doe'  # yahan par name by defualt deya gaya hai agr koi num pass nai karain gay to ya name aayega gaa
    signup_ts: datetime | None  
    tastes: dict[str, PositiveInt]  


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'tastes': {
        'wine': 9,
        b'cheese': 7,  
        'cabbage': '1',  
    },
}

user = User(**external_data)  

print(user.id)  
#> 123
display(user.model_dump())  



123


{'id': 123,
 'name': 'John Doe',
 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}

In [5]:
# continuing the above example...

from pydantic import ValidationError


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 'not an int', 'tastes': {}}  # yahan data galat diya gaya jis ki wajah se error aa raha hai

User(**external_data)  




ValidationError: 2 validation errors for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not an int', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing
signup_ts
  Field required [type=missing, input_value={'id': 'not an int', 'tastes': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [7]:

# oper jo error aaya iska solution 

try:
    User(**external_data)  
except ValidationError as e:
    display(e.errors())

[{'type': 'int_parsing',
  'loc': ('id',),
  'msg': 'Input should be a valid integer, unable to parse string as an integer',
  'input': 'not an int',
  'url': 'https://errors.pydantic.dev/2.5/v/int_parsing'},
 {'type': 'missing',
  'loc': ('signup_ts',),
  'msg': 'Field required',
  'input': {'id': 'not an int', 'tastes': {}},
  'url': 'https://errors.pydantic.dev/2.5/v/missing'}]

In [1]:
from pydantic import BaseModel, EmailStr, validator


In [13]:
from pydantic import BaseModel, EmailStr, validator

class User(BaseModel):
    name: str
    email: EmailStr
    account_id: int

    @validator('account_id')    # pydantic se validator ko import karay gay but jb isko istimal karna hoga to shoro main @ ka sign lagaye gay
    def validate_account_id(cls, value:int):

        if value <= 0:
            raise ValueError(f"Account ID must be Positive: {value}")
        return value
    
user1: User = User(name="AliRaza", email="AliRaza@gmail.org", account_id=200)


user_json_str: str = user1.model_dump_json()
print(user_json_str, type(user_json_str))

user_obj : User = User.model_validate_json(user_json_str)
display(user_obj)

{"name":"AliRaza","email":"AliRaza@gmail.org","account_id":200} <class 'str'>


C:\Users\Ali Raza\AppData\Local\Temp\ipykernel_12988\3747295865.py:8: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @validator('account_id')    # pydantic se validator ko import karay gay but jb isko istimal karna hoga to shoro main @ ka sign lagaye gay


User(name='AliRaza', email='AliRaza@gmail.org', account_id=200)

In [6]:
# is line main kuch b galat information dete jese email k bad .com nai lagaya ya phr name main num de diye to error aayega gaa

user1: User = User(name="Raza", email="raza@panacloud", account_id=200) # is line main email main .com ya .org nai lagaya

user1: User = User(name= 5 , email="raza@panacloud.org", account_id=200) # is line main b error hai kyun k name main str ki jagah int lagaya hai


ValidationError: 1 validation error for User
name
  Input should be a valid string [type=string_type, input_value=5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type

# Create a Model:


In [7]:
class User(BaseModel):
    name: str
    email: EmailStr
    account_id: int

    def validate_account_id(cls, value:int):
        if(value <= 0):
            raise ValueError(f"Account ID must be Positive: {value}")
        return value


In [11]:
user: User = User(name="Ali", email="ali@gmail.com", account_id=334)
print(user)


name='Ali' email='ali@gmail.com' account_id=334


In [14]:
display(user.model_dump_json())
user1.model_dump_json()


'{"name":"Ali","email":"ali@gmail.com","account_id":334}'

'{"name":"AliRaza","email":"AliRaza@gmail.org","account_id":200}'

In [18]:
User.model_validate_json(user.model_dump_json())


User(name='AliRaza', email='AliRaza@gmail.org', account_id=200)

In [19]:
User.model_validate_json(user1.model_dump_json())


User(name='AliRaza', email='AliRaza@gmail.org', account_id=200)

# JSON Output:


In [20]:
user_json_str: str = user.model_dump_json()
print(user_json_str, type(user_json_str))


{"name":"Ali","email":"ali@gmail.com","account_id":334} <class 'str'>


# Dictionary Object:


In [21]:
user_dict: dict = user.model_dump()
print(user_dict, type(user_dict))


{'name': 'Ali', 'email': 'ali@gmail.com', 'account_id': 334} <class 'dict'>


# Convert JSON str into Object:


In [22]:
user_obj : User = User.model_validate_json(user_json_str)
print(user_obj)


name='Ali' email='ali@gmail.com' account_id=334


In [23]:
from datetime import datetime
from typing import Optional

class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: list[int] = []

external_data: dict = {'id': '123', 'signup_ts': '2023-06-01 12:22', 'friends': [1, '2', b'3']}
user: User = User(**external_data)
print(user)
print(user.id)


id=123 name='John Doe' signup_ts=datetime.datetime(2023, 6, 1, 12, 22) friends=[1, 2, 3]
123
